# SWEN90006 Tutorial 11 Solution

## Your Tasks


### Question 1
Consider the function foo in Figure C.4 for checking the values of a given string (i.e. an array of characters). Assume that we run a *dynamic symbolic execution* (DSE) tool on this program, with the initial test being `arr = "xxxx"` and `count = 4`. 

Specify the path constraint that this will generate. Note that count is a constant.

```c
    1. int foo(char *arr, const int count) {
    2.   int i = 0; 
    3.   for (i = 0; i < count; i++) {
    4.     if (arr[i] == 'A' + i) return i;
    5.   }
         //0x4d4f4f42 is the 4-byte value of the string
         //"BOOM" stored in little-endian memory 
    6.   if (*(int*)(arr) != 0x4d4f4f42) {
    7.     return i;
    8.   }
    9.   return 0;
   10. }

```

<p style="text-align: center;">Figure C.4: The foo function</p>


**Solution:**

The loop in the function can be "unrolled" so that we can easily identify the path traversed by the given input. Since cound is a constant and its value is 4, we can rewrite the for-loop using 4 if statements as following.

```c
    1. int foo(char *arr, const int count) {
    2.   int i = 0;
    3.   if (i < count) {
    4.     if (arr[i] == 'A' + i)
    5.       return i;
    6.     i++;
    7.   }

    8.   if (i < count) {
    9.     if (arr[i] == 'A' + i)
   10.       return i;
   11.     i++;
   12.   }

   13.   if (i < count) {
   14.     if (arr[i] == 'A' + i)
   15.       return i;
   16.     i++;
   17.   }

   18.   if (i < count) {
   19.     if (arr[i] == 'A' + i)
   20.       return i;
   21.     i++;
   22.   }

   23.   if (*(int*)(arr) != 0x4d4f4f42)
   24.     return i;
   25.   return 0;
   26. }
```

The given input follows the path through lines 2-3-4-6-8-9-11-13-14-16-18-19-21-23-25. Suppose that arr[0] = S0, arr[1] = S1, arr[2] = S2, arr[3] = S3, the corresponding path constraint is: S0 != 'A', S1 != 'B', S2 != 'C', S3 != 'D', S0S1S2S3 != "BOOM".

### Question 2

Given the same program and the same input in Question 1, what test cases could be generated by a DSE tool?

**Solution:**

A "naive" DSE tool could generate 16 more test cases by "negating" all combination of branches on the path constraint in Question 1. All the 16 generated test cases are: "Axxx", "xBxx", "xxCx", "xxxD", "ABxx", "xBCx", "xxCD", "AxCx", "xBxD", "AxxD", "ABCx", "ABxD", "AxCD", "xBCD", "ABCD", and "BOOM".


However, effective DSE tools (e.g., SymCC, SAGE) take into account the code coverage (e.g., branch coverage or path coverage) of the program under test. For instance, it would not generate a string "ABxx" because it does not cover any new branches in addition to what "Axxx" has already covered on the control flow graph of this program.

Specifically, SymCC could only generate 5 more test cases: "Axxx", "xBxx", "xxCx", "xxxD", and "BOOM". It might apply some heuristics to reduce the number of paths to cope with the path explosion problem as well. See this discussion for more details: https://github.com/eurecom-s3/symcc/issues/88

Please follow the commands below to set up and run SymCC on the example program.

Build a Docker image for SymCC

```bash
git clone https://github.com/SWEN90006-2021/symcc.git
cd symcc
docker build . -t symcc
```

Run commands inside the docker container

```bash
docker run -it symcc
#inside the docker container
mkdir results
export SYMCC_OUTPUT_DIR=`pwd`/results
wget https://raw.githubusercontent.com/SWEN90006-2021/security-testing/main/dse_example.c
symcc dse_example.c -o dse_example
printf "xxxx" > seed
SYMCC_INPUT_FILE=seed ./dse_example seed

#all test cases will be generated and saved into the results folder
#the loop-unrolled version of the example is available at https://raw.githubusercontent.com/SWEN90006-2021/security-testing/main/dse_loop_unrolled_example.c
```